In [1]:
from lxml import etree 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
#%matplotlib qt

In [2]:
filename = "../data/data.xodr"

## Read XMl File

In [3]:
with open(filename) as fi: 
    xml_root = etree.parse(fi).getroot()

In [4]:
def save_lanes_as_dict(lanes): 
    lanes_dict = {
        'laneOffset': None,
        'laneSection': None, 
    }
    
    lane_section_dict = {
        's': None,
        'left': None, 
        'center': None, 
        'right': None,
    }
    left_dict = {
        'lane': None
    }
    
    link_dict = {
        'predecessor': None,
        'successor': None
    }
    
    lane_dict = {
        'id': None, 
        'type': None, 
        'level': None, 
        'link': None,
        'width': None, 
        'roadMark': None, 
        'userData': None
    }
    
    for child in lanes: 
        if (child.tag == 'laneOffset'):
            lanes_dict['laneOffset'] = {
                's' : child.get('s'),
                'a' : child.get('a'),
                'b' : child.get('b'),
                'c' : child.get('c'),
                'd' : child.get('d')
            }
            
        elif (child.tag == 'laneSection'): 
            lane_section_dict['s'] = child.get('s')
            left_lanes = []
            center_lanes = []
            right_lanes = []
            for direction in child: 
                for lane in direction:
                    widths = []

                    lane_dict['id'] = lane.get('id')
                    lane_dict['type'] = lane.get('type')
                    lane_dict['level'] = lane.get('level')
                    
                    ## iterate through lanes
                    for child in lane: 
                     
                        #Parse all links in lanes
                        if (child.tag == 'link'): 
                            for link in child: 
                                if (link.tag == 'predecessor'): 
                                    link_dict['predecessor'] = {
                                        'id': link.get('id')
                                    }
                                elif (link.tag == 'successor'): 
                                    link_dict['successor'] = {
                                        'id': link.get('id')
                                    }
                            lane_dict['link'] = link_dict
                        elif(child.tag == 'width'): 
                            widths.append({
                                'sOffset': child.get('sOffset'), 
                                'a': child.get('a'), 
                                'b': child.get('b'), 
                                'c': child.get('c'),
                                'd': child.get('d')
                            })
                        
                    
                    lane_dict['width'] = widths
                    if (direction.tag == 'left'): 
                        left_lanes.append(lane_dict)
                    elif(direction.tag == 'center'): 
                        center_lanes.append(lane_dict)
                    elif(direction.tag=='right'): 
                        right_lanes.append(lane_dict)
                    
                lane_section_dict['left'] = left_lanes
                lane_section_dict['center'] = center_lanes
                lane_section_dict['right'] = right_lanes
    
    lanes_dict['laneSection'] = lane_section_dict
    return lanes_dict

In [5]:
def save_links_as_dict(link): 
    link_dict = {
        'predecessor': None, 
        'successor': None
    }
    for child in link: 
        if (child.tag == 'predecessor'): 
            link_dict['predecessor'] = {
                'elementType': child.get('elementType'),
                'elementId': child.get('elementId'),
                'contactPoint': child.get('contactPoint')
            }
        elif (child.tag == 'successor'): 
            link_dict['successor'] = {
                'elementType': child.get('elementType'),
                'elementId': child.get('elementId'),
                'contactPoint': child.get('contactPoint')
            }
    
    return link_dict

In [6]:
def save_type_as_dict(road_type): 
    type_dict = {
       's': None, 
       'type': None, 
       'speed': None
    }
        
    speed_dict = {
        'max': None, 
        'unit': None
    }
    
    type_dict['s'] = road_type.get('s')
    type_dict['type'] = road_type.get('type')
    speed_dict['max'] = road_type[0].get('max')
    speed_dict['unit'] = road_type[0].get('unit')  
    type_dict['speed'] = speed_dict
    

In [7]:
def save_planView_as_dict(planView): 
    geometries = []
    for child in planView: 
        if (child.tag=='geometry'): 
            line = None
            arc = {
                'curvature': None
            }
            if (child[0].tag == 'line'): 
                line = True
            elif(child[0].tag == 'arc'): 
                arc['curvature'] = child[0].get('curvature')
            geometries.append({
                's': child.get('s'),
                'x': child.get('x'),
                'y': child.get('y'),
                'hdg': child.get('hdg'),
                'length': child.get('length'),
                'line': line, 
                'arc': arc
            })
    return geometries

In [8]:
def save_road_as_dict(road): 
    road_dict = {
        'name': road.get('name'), 
        'length': road.get('length'),
        'id': road.get('id'), 
        'junction': road.get('junction'),
        'link': None, 
        'type': None,
        'planView': None, 
        'elevationProfile': None, 
        'lateralProfile': None, 
        'lanes': None
    }
    for child in road: 
        if (child.tag == 'link'): 
            road_dict['link'] = save_links_as_dict(child)
        
        elif (child.tag == 'type'): 
            road_dict['type'] = save_type_as_dict(child)
                
        elif (child.tag == 'planView'): 
            road_dict['planView'] = save_planView_as_dict(child)
        #elif (child.tag == 'elevationProfile'): 
            
        #elif (child.tag == 'lateralProfile'): 
        
        elif (child.tag == 'lanes'): 
            road_dict['lanes'] = save_lanes_as_dict(child)
    
    return road_dict



In [9]:
all_roads = {}


for child in xml_root: 
    if (child.tag == 'road'): 
        current_road = save_road_as_dict(child)
        all_roads[int(current_road['id'])] = current_road
   
    


In [10]:
all_roads

{0: {'name': 'Road 0',
  'length': '1.3310253693587601e+1',
  'id': '0',
  'junction': '-1',
  'link': {'predecessor': {'elementType': 'road',
    'elementId': '3',
    'contactPoint': 'end'},
   'successor': {'elementType': 'road',
    'elementId': '10',
    'contactPoint': 'start'}},
  'type': None,
  'planView': [{'s': '0.0000000000000000e+0',
    'x': '1.0468000030517578e+2',
    'y': '9.3699998855590820e+0',
    'hdg': '1.5639764844735413e+0',
    'length': '7.9565878054869899e-1',
    'line': True,
    'arc': {'curvature': None}},
   {'s': '7.9565878054869899e-1',
    'x': '1.0468542653053798e+2',
    'y': '1.0165640163035381e+1',
    'hdg': '1.5639764844735451e+0',
    'length': '5.1838204616061470e+0',
    'line': None,
    'arc': {'curvature': '2.0000000000000000e-3'}},
   {'s': '5.9794792421548459e+0',
    'x': '1.0469390733236408e+2',
    'y': '1.5349430470683362e+1',
    'hdg': '1.5743441253967780e+0',
    'length': '6.7218615357127487e-1',
    'line': True,
    'arc': {'cu

In [11]:
df = pd.DataFrame.from_dict(all_roads, orient='index')

In [12]:
df

,name,length,id,junction,link,type,planView,elevationProfile,lateralProfile,lanes
0,Road 0,1.3310253693587601e+1,0,-1,"{'predecessor': {'elementType': 'road', 'eleme...",None,"[{'s': '0.0000000000000000e+0', 'x': '1.046800...",None,None,"{'laneOffset': {'s': '0.0000000000000000e+0', ..."
1,Road 1,1.1992999999999999e+2,1,-1,"{'predecessor': {'elementType': 'road', 'eleme...",None,"[{'s': '0.0000000000000000e+0', 'x': '1.935999...",None,None,"{'laneOffset': {'s': '0.0000000000000000e+0', ..."
2,Road 2,1.1080000000000013e+1,2,-1,"{'predecessor': {'elementType': 'junction', 'e...",None,"[{'s': '0.0000000000000000e+0', 'x': '1.042528...",None,None,"{'laneOffset': {'s': '0.0000000000000000e+0', ..."
3,Road 3,4.9083880777828881e+0,3,-1,"{'predecessor': {'elementType': 'junction', 'e...",None,"[{'s': '0.0000000000000000e+0', 'x': '1.046465...",None,None,"{'laneOffset': {'s': '0.0000000000000000e+0', ..."
4,Road 4,2.8370000000000001e+1,4,-1,"{'predecessor': {'elementType': 'road', 'eleme...",None,"[{'s': '0.0000000000000000e+0', 'x': '-1.79999...",None,None,"{'laneOffset': {'s': '0.0000000000000000e+0', ..."
...,...,...,...,...,...,...,...,...,...,...
900,Road 900,2.5613483531034351e+1,900,895,"{'predecessor': {'elementType': 'road', 'eleme...",None,"[{'s': '0.0000000000000000e+0', 'x': '-4.33239...",None,None,"{'laneOffset': {'s': '2.5603483531034332e+1', ..."
933,Road 933,3.0920000000000009e+1,933,895,"{'predecessor': {'elementType': 'road', 'eleme...",None,"[{'s': '0.0000000000000000e+0', 'x': '-4.68239...",None,None,"{'laneOffset': {'s': '2.7740000000000002e+1', ..."
934,Road 934,3.0920000000000009e+1,934,895,"{'predecessor': {'elementType': 'road', 'eleme...",None,"[{'s': '0.0000000000000000e+0', 'x': '-4.68239...",None,None,"{'laneOffset': {'s': '2.7740000000000002e+1', ..."
939,Road 939,2.6488131460062810e+1,939,895,"{'predecessor': {'elementType': 'road', 'eleme...",None,"[{'s': '0.0000000000000000e+0', 'x': '-2.70198...",None,None,"{'laneOffset': {'s': '2.3308131460062803e+1', ..."


In [13]:
geometries = {}
for index, row in df.iterrows(): 
    
    geometries[index] = row['planView']

In [14]:
geometries

{0: [{'s': '0.0000000000000000e+0',
   'x': '1.0468000030517578e+2',
   'y': '9.3699998855590820e+0',
   'hdg': '1.5639764844735413e+0',
   'length': '7.9565878054869899e-1',
   'line': True,
   'arc': {'curvature': None}},
  {'s': '7.9565878054869899e-1',
   'x': '1.0468542653053798e+2',
   'y': '1.0165640163035381e+1',
   'hdg': '1.5639764844735451e+0',
   'length': '5.1838204616061470e+0',
   'line': None,
   'arc': {'curvature': '2.0000000000000000e-3'}},
  {'s': '5.9794792421548459e+0',
   'x': '1.0469390733236408e+2',
   'y': '1.5349430470683362e+1',
   'hdg': '1.5743441253967780e+0',
   'length': '6.7218615357127487e-1',
   'line': True,
   'arc': {'curvature': None}},
  {'s': '6.6516653957261207e+0',
   'x': '1.0469152255627105e+2',
   'y': '1.6021612393897556e+1',
   'hdg': '-4.7088411817828195e+0',
   'length': '6.5280571510994942e+0',
   'line': None,
   'arc': {'curvature': '-2.8022338979856666e-3'}},
  {'s': '1.3179722546825616e+1',
   'x': '1.0472807096528994e+2',
   'y':

In [15]:
def calculate_line(length, hdg, x,y):      
    return lambda t: (np.array([np.cos(hdg), np.sin(hdg)]) * length * t) + np.array([x,y])

def calculate_arc(length, hdg, curvature, x,y): 
    return lambda t: np.array([x,y]) + 1/curvature * np.array([np.cos(t/(1/curvature)), np.sin(t/(1/curvature))])

def deriv_arc(length, hdg, curvature, x,y): 
    return lambda t: np.array([-np.sin(t*curvature), np.cos(t*curvature)])

def deriv_line(length, hdg): 
    return lambda t: np.array([-np.sin(hdg), np.cos(hdg)])*length

In [16]:
step = 0.001
line_points = []
arc_points = []
line = calculate_line(2, np.pi/3, 0,0)




#arc = calculate_arc(0.35, np.pi, 0.5, 4,4)
#for t in np.arange(0, 2*np.pi + step, step): 
#    arc_points.append(arc(t))
    
#arc_points = np.array(arc_points)

In [17]:
def get_adjusted_arc(x,y,hdg, length, curvature):
    deriv_of_arc = deriv_arc(length, hdg, curvature, x,y)
    deriv_of_line = deriv_line(length, hdg)
    tempo_vec = deriv_of_line(0)
    line_gradient = tempo_vec[1] / tempo_vec[0]
    
    step = 0.01
    
    arc_Ts = np.arange(0, 2*np.pi + step, step)
    
    arc_gradients = []
    line_gradients = []
    for t in arc_Ts: 
        arc_tempo_vec = deriv_of_arc(t)
        if (arc_tempo_vec[0] == 0): 
            arc_tempo_vec[0] = 0.00000000001
        arc_gradients.append(arc_tempo_vec[1] / arc_tempo_vec[0])
        line_gradients.append(line_gradient)
    
    gradient_diffs = abs(np.array(arc_gradients)-np.array(line_gradients))
    min_diff_index = np.argmin(gradient_diffs)
    t = arc_Ts[min_diff_index]
    
    arc_Ts = np.arange(t, 1000 + step, step)
    arc = calculate_arc(length, hdg, curvature, x,y)
    adjusted_arc_points = []
    arc_length = 0
    # Do the parametrisation and translate all points so that start of the curve is at x,y
    for t in arc_Ts: 
        adjusted_arc_points.append(arc(t))
        arc_length += np.linalg.norm(deriv_of_arc(t)*step)
        
        if (arc_length >= length): 
            break
        
    
    # translate points
    adjusted_arc_points = np.array(adjusted_arc_points)
    difference_vec = np.array(adjusted_arc_points[0] - np.array([x,y])) 
    adjusted_arc_points = adjusted_arc_points - difference_vec
    return adjusted_arc_points
    

In [18]:
arc_points = get_adjusted_arc(0,0, np.pi/3, 2, -.2)

In [19]:
%matplotlib qt
plt.plot(arc_points[:,0], arc_points[:,1])
plt.scatter(arc_points[0:3,0], arc_points[0:3,1], c="orange")
plt.gca().set_aspect('equal')
plt.grid(True)
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.show()

In [34]:
def get_all_geoms_as_points(): 
    plot_lines = []
    plot_arcs = np.empty([0,2])
    step = 0.001
    Line_Ts = np.arange(0,1+step,step)
    Arc_Ts = np.arange(0, 2*np.pi + step, step)
    for geometry in geometries: 
        current = geometries[geometry] 
        for g in current: 
            #print(g)
            if (g['line']): 
                #print("Line")
                line = calculate_line(float(g['length']), float(g['hdg']), float(g['x']),float(g['y']))
                points = [line(t) for t in Line_Ts]
                plot_lines += points
            elif (g['line'] != True and g['arc']['curvature'] != None): 
                #print(g['arc']['curvature'])
                #arc = calculate_arc(float(g['length']), float(g['hdg']), float(g['arc']['curvature']),float(g['x']),float(g['y']))
                points = get_adjusted_arc(float(g['length']), float(g['hdg']), float(g['arc']['curvature']),float(g['x']),float(g['y']))
                plot_arcs = np.concatenate([plot_arcs, points])  
    plot_lines = np.array(plot_lines)
    plot_arcs = np.array(plot_arcs)
    return plot_lines, plot_arcs

In [35]:
lines, arcs = get_all_geoms_as_points()


In [36]:
arcs

array([[ 5.18382046,  1.56397648],
       [ 5.18438477,  1.55399673],
       [ 5.18595893,  1.54412577],
       ...,
       [10.93246546, -3.76852539],
       [ 7.28974406, -3.14287127],
       [ 7.33545228, -3.94845782]])

In [42]:
#plt.scatter(lines[:,0],lines[:,1], c='blue', label="Lines",s=0.1)
plt.scatter(arcs[0:100,0],arcs[0:100,1], c='orange', label="Arcs", s= 1)
plt.show()

In [71]:
plt.plot(line_points[:,0], line_points[:,1])

#plt.plot(arc_points[:,0], arc_points[:,1])
plt.gca().set_aspect('equal')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.show()

In [17]:
x = []
y = []

for road in geometries: 
    for geometrie in geometries[road]: 
        x.append(float(geometrie['x']))
        y.append(float(geometrie['y']))

In [18]:
plt.scatter(x,y)

In [128]:
np.concatenate([np.empty([0,2]), np.zeros([20,2])])

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])